<a href="https://colab.research.google.com/github/gabrielgranados97/Procesamiento-de-lenguaje-natural-2022/blob/main/Primera_tarea_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Primera tarea

**Lectura del set de datos**

In [36]:
import numpy as np
import os
import gzip

def load_set(ruta, tipo='train'):

  ruta_categorias = os.path.join(ruta, '%s-labels-idx1-ubyte.gz' % tipo)
  ruta_imagenes = os.path.join(ruta, '%s-images-idx3-ubyte.gz' % tipo)
    
  with gzip.open(ruta_categorias, 'rb') as rut_cat:
      etiquetas = np.frombuffer(rut_cat.read(), dtype=np.uint8, offset=8)

  with gzip.open(ruta_imagenes, 'rb') as rut_imgs:
      imagenes = np.frombuffer(rut_imgs.read(), dtype=np.uint8, offset=16).reshape(len(etiquetas), 784)

  return imagenes, etiquetas

In [37]:
from google.colab import drive
drive.mount('/content/gdrive')

ruta = '/content/gdrive/MyDrive/Colab Notebooks/fashion_mnist_data'

X_train, Y_train = load_set(ruta, tipo='train')
X_test, Y_test = load_set(ruta, tipo='test')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


**Reajustar el tamaño de los datos**

In [38]:
X_train = X_train[0:59904:]
X_test = X_test[0:9984:]
Y_train = Y_train[0:59904:]
Y_test = Y_test[0:9984]

X_train = np.reshape(X_train,(59904,28,28,1))
X_test = np.reshape(X_test,(9984,28,28,1))

**Importar tensorflow 2 (ya incluye Keras)**

In [39]:
%tensorflow_version 2.x
import tensorflow as tf

print('Versión de Tensorflow: ' + tf.__version__)


Versión de Tensorflow: 2.8.0


**Verificar disponibilidad de la TPU**

In [40]:
from tensorflow.python.distribute import tpu_strategy
from six import print_
from tensorflow.python.ops.gen_logging_ops import Print
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
  print('TPU Encontrada ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: TPU no encontrada')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)  

TPU Encontrada  ['10.111.251.26:8470']
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.111.251.26:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.111.251.26:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


**Implementar el modelo usando TPU **

In [44]:
tf.random.set_seed(200)

with tpu_strategy.scope ():
  modelo = tf.keras.models.Sequential()
  modelo.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
  modelo.add(tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='elu'))
  modelo.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  modelo.add(tf.keras.layers.Dropout(0.25))

  modelo.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
  modelo.add(tf.keras.layers.Conv2D(128, (5, 5), padding='same', activation='elu'))
  modelo.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
  modelo.add(tf.keras.layers.Dropout(0.25))

  modelo.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
  modelo.add(tf.keras.layers.Conv2D(256, (5, 5), padding='same', activation='elu'))
  modelo.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  modelo.add(tf.keras.layers.Dropout(0.25))

  modelo.add(tf.keras.layers.Flatten())
  modelo.add(tf.keras.layers.Dense(256))
  modelo.add(tf.keras.layers.Activation('elu'))
  modelo.add(tf.keras.layers.Dropout(0.5))
  modelo.add(tf.keras.layers.Dense(10))
  modelo.add(tf.keras.layers.Activation('softmax'))

#Compilar el modelo e imprimir información en pantalla

  modelo.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
# checkpoint
  cp_path='/content/gdrive/MyDrive/Colab Notebooks'
  tpu_callback =tf.keras.callbacks.ModelCheckpoint(cp_path,save_best_only=True,save_weights_only=True,verbose=1)

  modelo.summary()


Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_36 (Bat  (None, 28, 28, 1)        4         
 chNormalization)                                                
                                                                 
 conv2d_36 (Conv2D)          (None, 28, 28, 64)        1664      
                                                                 
 max_pooling2d_36 (MaxPoolin  (None, 14, 14, 64)       0         
 g2D)                                                            
                                                                 
 dropout_48 (Dropout)        (None, 14, 14, 64)        0         
                                                                 
 batch_normalization_37 (Bat  (None, 14, 14, 64)       256       
 chNormalization)                                                
                                                     

**Entrenamiento con TPU**


In [45]:
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)
Y_train = Y_train.astype(np.float32)
Y_test = Y_test.astype(np.float32)

import timeit

def entrenamiento_tpu():
  modelo.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=128,callbacks=[tpu_callback], epochs=2,verbose=1)
  return None
  
tpu_time = timeit.timeit('entrenamiento_tpu()', number=1, setup='from __main__ import entrenamiento_tpu')

Epoch 1/2
467/468 [============================>.] - ETA: 0s - loss: 0.7224 - accuracy: 0.7710
Epoch 1: val_loss improved from inf to 0.36439, saving model to /content/gdrive/MyDrive/Colab Notebooks
468/468 [==============================] - 17s 25ms/step - loss: 0.7222 - accuracy: 0.7710 - val_loss: 0.3644 - val_accuracy: 0.8726
Epoch 2/2
467/468 [============================>.] - ETA: 0s - loss: 0.3997 - accuracy: 0.8587
Epoch 2: val_loss improved from 0.36439 to 0.34081, saving model to /content/gdrive/MyDrive/Colab Notebooks
468/468 [==============================] - 10s 21ms/step - loss: 0.3994 - accuracy: 0.8588 - val_loss: 0.3408 - val_accuracy: 0.8880


In [46]:
print('Tiempo de entrenamiento ' + str(tpu_time)+ ' segundos')

Tiempo de entrenamiento 29.231464975000563 segundos
